In [1]:
import os

In [2]:
%pwd

'/Users/prathmesh/Desktop/chicken_disease-cnnclassifier/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/prathmesh/Desktop/chicken_disease-cnnclassifier'

In [5]:
%cd chicken_disease-cnnclassifier

[Errno 2] No such file or directory: 'chicken_disease-cnnclassifier'
/Users/prathmesh/Desktop/chicken_disease-cnnclassifier/research


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """Data Ingestion Configuration"""
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [7]:
from cnnclassifier.constant import *
from cnnclassifier.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

        

In [9]:
import os
import urllib.request as request
import zipfile
from cnnclassifier import logger
from cnnclassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}.")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted all the files to {unzip_path}")

In [11]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-08-27 23:03:39,240: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-27 23:03:39,241: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-27 23:03:39,242: INFO: common: created directory at: artifacts]
[2025-08-27 23:03:39,242: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-27 23:03:39,242: INFO: 3874346788: File already exists of size: ~ 11349 KB.]
[2025-08-27 23:03:39,411: INFO: 3874346788: Extracted all the files to artifacts/data_ingestion]
